In [ ]:
!pip install bertopic
!pip install wget
!pip install transformers
!pip install natasha
!pip install hdbscan

In [ ]:
!git clone https://github.com/VikaKirichenko/HSE_shop.git

In [ ]:
import pandas as pd
# loading testdata (4500 items, 5 categories)
df = pd.read_csv("group_project_shops/data/products.csv")
df = df.rename(columns = {"Наименование":"item_name"})
df = df[['item_name',"category"]]

In [ ]:
df.head()

,item_name,category
0,✅ Развивающие книги для детей развивашки тетра...,Детское творчество и досуг
1,"Развивающие книги для детей, тетрадь развивашк...",Детское творчество и досуг
2,✅ Развивающие книги для детей развивашки тетра...,Детское творчество и досуг
3,Развивающие книги для детей пиши стирай,Детское творчество и досуг
4,"Развивающие книги для детей, развивашки",Детское творчество и досуг


Without own embeddings.

Embedding model =  SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [ ]:
import HSE_shop.topics.extract_topics as extract_topics

topic_model, topics, probs = extract_topics.create_and_train_model(df['item_name'].tolist(),n_topics=15)

Batches:   0%|          | 0/141 [00:00<?, ?it/s]

2023-03-16 20:11:35,444 - BERTopic - Transformed documents to Embeddings
2023-03-16 20:11:50,659 - BERTopic - Reduced dimensionality
2023-03-16 20:11:54,875 - BERTopic - Clustered reduced embeddings
2023-03-16 20:13:37,296 - BERTopic - Reduced number of topics from 144 to 15


In [ ]:
print("collected topics \n", topics)

collected topics 
 [-1, 2, -1, 2, 2, -1, 2, -1, 2, 2, 2, 2, 2, 2, 2, -1, -1, -1, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, -1, 2, -1, 2, 2, -1, 2, 0, -1, 2, -1, 2, 2, 2, 2, -1, 0, -1, -1, 0, 2, 2, 2, 2, -1, 2, 2, 11, 2, -1, 2, 0, -1, -1, -1, -1, 2, -1, 0, 2, 2, 2, -1, 2, 2, 0, 2, 2, 2, -1, -1, 2, 2, -1, 2, 2, 2, 0, 2, 0, 11, 2, 0, -1, 0, 9, 2, 2, 2, -1, 0, -1, 2, -1, 2, -1, 0, 2, 1, 11, 2, -1, 2, 2, 0, -1, 2, 0, 2, -1, 2, 0, -1, -1, 2, 2, 2, 0, 0, 0, -1, 2, 2, 2, 2, -1, 3, 2, 2, 0, 2, 9, -1, 2, 2, -1, 2, 2, -1, 2, 2, 2, 2, -1, 2, -1, -1, -1, 2, 2, 0, 0, 2, 2, 2, -1, 2, 0, 2, 0, -1, 2, 2, -1, -1, 2, 2, 2, 9, -1, 2, 2, 2, 2, 2, 0, 0, -1, 0, 2, -1, 2, 2, -1, 2, 0, -1, 0, 2, 2, 2, 2, -1, 2, -1, -1, 2, 2, 2, -1, 2, 2, -1, -1, -1, 2, -1, 2, 2, 4, 2, 0, 4, 2, 2, 9, -1, 2, -1, -1, 2, 12, 2, -1, 2, 0, 0, 0, 0, -1, 2, 2, 2, -1, 2, 2, 2, 0, -1, -1, 2, 2, 2, 2, 0, -1, 0, 2, -1, 2, 2, 2, -1, 2, 2, -1, -1, -1, 2, 2, 2, -1, 2, -1, -1, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, -1, 0, 0, -1, 2, -1, 2, -1, 0, 2, -1

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(top_n_topics=15)

In [ ]:
pred = extract_topics.predict_topic(topic_model, "Развивающие книги для детей пиши стирай", n_count=3)
print("top topics to predic:",pred)

top topics to predic: ['2_книги для_набор книг_детская энциклопедия', '-1_для детей_для рыбы_нож для', '3_для макияжа_расческа для_для волос']


Test model with bert embeddings

In [ ]:
from HSE_shop.vectorizing import vectorizing_funcs

df['item_name_norm'] = df[['item_name']].applymap(tokenize_lemmatize_text)
input_ids, attention_masks = vectorizing_funcs.get_bert_vector(df['item_name_norm'].tolist())
topic_model, topics, probs = extract_topics.create_and_train_model(df['item_name'].tolist(),n_topics=15, embeddings = input_ids.numpy())


2023-03-16 20:44:43,841 - BERTopic - Reduced dimensionality
2023-03-16 20:44:49,318 - BERTopic - Clustered reduced embeddings
2023-03-16 20:44:49,491 - BERTopic - Reduced number of topics from 150 to 15


In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart()

Test model with tfidf embeddings

In [ ]:
from HSE_shop.vectorizing.vectorizing_funcs import get_tfidf_vector
from HSE_shop.transform.transform_data import tokenize_lemmatize_text

x_vector, vectorizer = get_tfidf_vector(df['item_name_norm'].tolist())

topic_model, topics, probs = extract_topics.create_and_train_model(df['item_name'].tolist(),embeddings = x_vector)

print("collected topics + \n", topics)

/usr/local/lib/python3.9/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.

2023-03-16 20:48:14,849 - BERTopic - Reduced dimensionality
2023-03-16 20:48:18,321 - BERTopic - Clustered reduced embeddings
2023-03-16 20:48:18,483 - BERTopic - Reduced number of topics from 130 to 10


collected topics + 
 [0, 0, 0, 0, 0, 5, 6, 0, -1, 0, 1, 0, 0, 6, -1, -1, 0, 5, 1, 0, 1, 0, 0, -1, 1, -1, -1, 0, -1, 6, -1, 0, 5, 1, -1, 0, -1, 0, 0, -1, 0, 0, -1, 1, 8, 3, 1, -1, 0, 0, 1, 7, -1, 1, 0, 0, 1, 3, -1, -1, 0, 7, 0, -1, -1, 1, -1, 8, 1, -1, 1, 4, 3, 6, 5, 1, -1, 3, 1, 1, 0, -1, -1, -1, -1, 0, 1, 0, 6, -1, 0, 2, -1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 2, 1, 2, 1, -1, -1, -1, 2, 1, -1, 1, -1, 0, -1, 0, -1, -1, -1, 6, 0, 1, 2, -1, 1, 6, 0, 1, 1, 2, -1, 1, 0, 1, 0, 1, -1, -1, 1, 1, 1, 6, 1, 1, -1, 1, 0, -1, -1, -1, 1, 0, 1, 1, -1, 0, 0, 0, -1, 2, 0, -1, 2, 4, 1, 5, -1, 0, 0, 1, -1, -1, 1, 7, 0, -1, -1, 2, -1, 4, 0, 2, 0, 2, 4, 1, 1, 2, 1, 2, -1, 1, 2, 0, 1, 2, 0, -1, 6, 8, 2, 8, -1, -1, 2, -1, 0, 2, 4, 2, 0, 1, 4, 1, -1, 0, 7, 0, 4, 1, 1, 4, 0, 8, 4, 4, 1, 0, -1, 0, -1, 0, 1, 0, -1, -1, -1, 1, 0, 1, 0, -1, 1, 0, -1, 0, -1, 0, -1, 0, 0, 0, 1, 2, -1, 1, -1, 1, 1, -1, 0, 1, 0, 0, 1, 1, -1, 0, 0, -1, 1, 0, 1, -1, 0, 0, 0, 0, -1, -1, 0, 7, -1, -1, 8, 1, -1, 6, 1, 0, 1, 5, 1, -1, 0, 1, -1, 0, 

In [ ]:
topic_model.visualize_barchart(top_n_topics=15)